<img src="https://cybersecurity-excellence-awards.com/wp-content/uploads/2017/06/366812.png">


<h1><center>Darwin Forecasting Model Building </center></h1>


# Prior to getting started:

Darwin notebook will no longer support 'Register User' starting from 2.0. As a user, you must have credentials ready before using this notebook. 

In order to proceed, in the Environment Variables cell: 
1. Set your username and password to ensure that you're able to log in successfully
2. Set the path to the location of your datasets if you are using your own data.  The path is set for examples.
  <br><b>NOTE:</b> We provide two ways to analyze feature importance. One is to use the entire dataset; the other one is to analyze a few samples to understand individual samples. In the latter case, we advise users to use a small dataset (<=500) because it takes long time to process individual samples. 

Here are a few things to be mindful of:
1. For every run, check the job status (i.e. requested, failed, running, completed) and wait for job to complete before proceeding. 
2. If you're not satisfied with your model and think that Darwin can benefit from extra training, use the resume function.

## Import Necessary Libraries

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
import datetime
from IPython.display import Image
from time import sleep
from amb_sdk.sdk import DarwinSdk

## Set Darwin SDK

In [ ]:
ds = DarwinSdk()
ds.set_url('https://darwin-api.sparkcognition.com/v1/')

## Environment Variables

In [ ]:
#Set your user id and password accordingly
USER="[your Darwin user id]"
PW="[your Darwin password]"

# Set path to datasets - The default below assumes Jupyter was started from amb-sdk/examples/Enterprise/
# Modify accordingly if you wish to use your own data
PATH_TO_DATASET = '../../sets/'
TRAIN_DATASET = 'sine_train.csv'
TEST_DATASET = 'sine_test.csv'

# A timestamp is used to create a unique name in the event you execute the workflow multiple times or with 
# different datasets.  File names must be unique in Darwin.
ts = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())

# User Login

In [ ]:
status, msg = ds.auth_login_user(USER, PW)
if not status:
    print(msg)
else:
    print('You are logged in.')

# Data Upload

**Read dataset and view a file snippet**

In [ ]:
df = pd.read_csv(os.path.join(PATH_TO_DATASET, TRAIN_DATASET))
df.head()

**Upload training dataset to Darwin**

In [ ]:
status, dataset = ds.upload_dataset(os.path.join(PATH_TO_DATASET, TRAIN_DATASET))
if not status:
    print(dataset)
else:
    print('Data is successfully uploaded!')

#### **Upload testing dataset to Darwin**

In [ ]:
status, dataset = ds.upload_dataset(os.path.join(PATH_TO_DATASET, TEST_DATASET))
if not status:
    print(dataset)
else:
    print('Data is successfully uploaded!')

# Analyze Data
Analyze data is a necessary step before cleaning data and creating model. 

In [ ]:
status, analyze_id = ds.analyze_data(TRAIN_DATASET, 
                                     job_name = 'Darwin_analyze_data_job' + "-" + ts, 
                                     artifact_name = 'Darwin_analyze_data_artifact' + "-" + ts)
sleep(1)
if status:
    ds.wait_for_job('Darwin_analyze_data_job' + "-" + ts)
else:
    print(analyze_id)

In [ ]:
ds.lookup_job_status_name(analyze_id['job_name'])

# Clean Data

Starting in Version 1.6, Darwin SDK offers a way to clean your data outside of model training. Every dataset needs to be cleaned before creating a model. There is no need to save the cleaned data and upload it, but users need to specify the target name before running.

In [ ]:
target = 'y'
status, job_id = ds.clean_data(dataset_name=TRAIN_DATASET, target=target)
if not status:
    print(job_id)
else:
    print('Data has been successfully cleaned!')

# Create and Train Model 

In the cell below, specify the parameters used to create the forecasting model:
- `model`: the name of your model
- `forecast_horizon`: the forecast length, which is how far you need to predict in the future, int type
- `max_train_time`: the amount of time used for training (possibly shorter with early stopping)

In [ ]:
ts = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())
model = "model" + "-" + ts
forecast_horizon = 2
max_train_time = '00:02'
status, job_id = ds.create_model(dataset_names=TRAIN_DATASET,
                                 model_name=model,
                                 forecast_horizon=forecast_horizon,
                                 fit_profile_name=job_id['profile_name'],
                                 max_train_time=max_train_time)
sleep(1)
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

In [ ]:
# look up job status
ds.lookup_job_status_name(job_id['job_name'])

In [ ]:
# look up the model details
model_info = ds.lookup_model_name(job_id['model_name'])

## Extra Training (Optional)
Run the following cell for extra training, specify the amount of time for extra training using `max_train_time` 

In [ ]:
max_train_time = '00:01'
status, job_id = ds.resume_training_model(dataset_names=TRAIN_DATASET,
                                          model_name=model,
                                          max_train_time=max_train_time)
sleep(1)
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

## Predict
Run the following cell for prediction

In [ ]:
# clean dataset
status, job_id = ds.clean_data(TEST_DATASET, model_name=model)

if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

In [ ]:
# Test model
status, artifact = ds.run_model(dataset_name=TEST_DATASET, 
                                model_name=model, 
                                forecast_horizon=forecast_horizon)
sleep(1)
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

In [ ]:
# Get predictions
status, prediction = ds.download_artifact(artifact['artifact_name'])

In [ ]:
# View prediction
prediction.head()

## Visualization

In [ ]:
# read in test dataset
df = pd.read_csv(os.path.join(PATH_TO_DATASET, TEST_DATASET))

In [ ]:
# plot against predictions
seq_len = model_info[1]['description']['best_genome'][0]['layer 1']['parameters']['seqlength']
preds = prediction.iloc[::forecast_horizon, :].values.reshape(-1)
plt.rcParams['figure.figsize']= 20,20
plt.plot(df[target][seq_len:].reset_index(drop=True), color='b')
plt.plot(preds, color='r')